# Этап 1

In [5]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import os
import pandas as pd
from tqdm import tqdm
import torch
import pyarrow.parquet as pq
import dask.dataframe as dd

# # Чтение Parquet-файла с использованием pyarrow
# table = pq.read_table('./reviews_keywords/wildberries_reviews_corrected.parquet')

# # Преобразование в pandas DataFrame
# df_pandas = table.to_pandas()

# # Преобразование pandas DataFrame в Dask DataFrame
# df_dask = dd.from_pandas(df_pandas, npartitions=100)  # Укажите количество нужных вам частей
# df_pandas = None
# table = None
# import gc
# gc.collect()
# df_dask

In [6]:
result = pd.read_csv("./reviews_keywords/wildberries_reviews.csv", nrows=1000)
result.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Unnamed: 0        1000 non-null   int64
 1   review_full_text  1000 non-null   object
 2   review_rating     1000 non-null   int64
 3   product           1000 non-null   object
 4   category          1000 non-null   object
 5   url               1000 non-null   object
 6   corrected_text    1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 540.9+ KB


In [7]:
# Оставляем только по 5 записей для каждого уникального значения в столбце 'product'
# result_limited = result.groupby('product').apply(lambda x: x.iloc[5:8]).reset_index(drop=True)
result_limited = result


In [8]:
import spacy
import re
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter
import nltk
from nltk.corpus import stopwords
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import hdbscan
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm

# Загрузка дообученной модели и токенизатора
# Загружаем конфигурацию модели


# Загрузка модели и токенайзера от Сбербанка
tokenizer = BertTokenizerFast.from_pretrained("./reviews_keywords/fine_tuned_model")
model = AutoModel.from_pretrained("./reviews_keywords/fine_tuned_model").to(device)
# Инициализируем модель с конфигурацией
config = BertConfig.from_pretrained('./reviews_keywords/fine_tuned_model', output_hidden_states=True)
model_classification = BertForSequenceClassification.from_pretrained('./reviews_keywords/fine_tuned_model', config=config).to(device)

        # self.tokenizer_my = BertTokenizerFast.from_pretrained(self.model_path)
        #  # Загрузка модели для классификации
        # self.classification_model = BertForSequenceClassification.from_pretrained(self.model_path).to(self.device)
        # # Загрузка базовой модели для получения эмбеддингов
        # self.embedding_model = AutoModel.from_pretrained(self.model_path).to(self.device)

spacy.prefer_gpu()
# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg", disable=["ner", "tagger", "attribute_ruler", "lemmatizer"])

df = result_limited

# Преобразование pandas DataFrame в Hugging Face Dataset
dataset = Dataset.from_pandas(df)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [9]:
# from tqdm import tqdm

# # Кастомный Dataset для DataLoader
# class SentencesDataset(torch.utils.data.Dataset):
#     def __init__(self, sentences, tokenizer, max_length=128):
#         self.sentences = sentences
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         # Токенизация строки
#         encoded = self.tokenizer(
#             self.sentences[idx],
#             return_tensors='pt',
#             padding='max_length',
#             truncation=True,
#             max_length=self.max_length
#         )
#         # Возвращаем input_ids и attention_mask
#         return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

# # Функция для обработки батчей и получения эмбеддингов
# def get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32):
#     dataset = SentencesDataset(sentences, tokenizer)
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
#     all_embeddings = []
#     model.eval()
    
#     with torch.no_grad():
#         for batch in dataloader:
#             input_ids, attention_mask = batch
#             input_ids = input_ids.to(device)
#             attention_mask = attention_mask.to(device)
            
#             outputs = model(input_ids, attention_mask=attention_mask)
#             embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
#             all_embeddings.extend(embeddings)
    
#     return np.array(all_embeddings)

In [10]:
def clean_text(text):
    text = re.sub(r'[\n\r\t]+|\s{2,}', ' ', text)  # Объединяем шаги для замены пробелов
    text = re.sub(r'(?<!\.)\s*\.\s*|\s*\.\s*(?!\.)', '. ', text)  # Оптимизация замены точки
    return text.strip().rstrip('.')

def split_by_syntax(doc, max_length=100):
    """
    Разделяет длинные предложения на более короткие, основываясь на синтаксическом анализе.
    """
    split_sentences = []
    current_chunk = []
    if len(doc.text) < 100:
        return [doc.text]
    for token in doc:
        current_chunk.append(token.text)
        
        # Проверка, можно ли завершить предложение
        if token.dep_ in ('punct', 'cc', 'conj') or len(' '.join(current_chunk)) > max_length:
            if len(' '.join(current_chunk)) > max_length:
                split_sentences.append(' '.join(current_chunk[:-1]))
                current_chunk = [current_chunk[-1]]
            else:
                split_sentences.append(' '.join(current_chunk))
                current_chunk = []

    # Добавление последнего оставшегося фрагмента
    if current_chunk:
        split_sentences.append(' '.join(current_chunk))

    return split_sentences

def split_reviews_into_sentences(batch):
    # Очистка текстов
    cleaned_texts = [clean_text(text) for text in batch['corrected_text']]
    
    # Обработка текстов с помощью nlp.pipe с указанием batch_size
    docs = list(nlp.pipe(cleaned_texts, batch_size=64))

    # Извлечение предложений и их дальнейшая обработка
    processed_sentences = []
    for doc in docs:
        sentences = []
        for sent in doc.sents:
            # Разделяем длинные предложения по синтаксису
            split_sentences = split_by_syntax(sent, max_length=100)
            sentences.extend(split_sentences)
        processed_sentences.append(sentences)
    
    batch['sentences'] = processed_sentences
    
    return batch

dataset = dataset.map(split_reviews_into_sentences, batched=True, batch_size=32)

# Преобразуем Dataset обратно в pandas DataFrame
df = dataset.to_pandas()

# Выполним explode по столбцу с предложениями
df_exploded = df.explode('sentences').reset_index(drop=True)

# Удаляем лишние столбцы, которые появились после explode
df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])

# Преобразуем DataFrame обратно в Hugging Face Dataset
dataset_exploded = Dataset.from_pandas(df_exploded)

from torch.cuda.amp import autocast

def compute_sentence_embeddings(sentences):
    # Фильтруем список, оставляя только строки
    sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not sentences:
        raise ValueError("Input contains no valid strings.")

    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        with autocast():  # Используем mixed precision для ускорения
            outputs = model(**inputs)
    
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

def compute_embeddings(batch, imput_col = 'sentences', output_col = 'sentence_embeddings'):
    sentences = batch[imput_col]

    # Проверяем, что все элементы в batch являются строками
    valid_sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not valid_sentences:
        batch[output_col] = [[]] * len(sentences)  # Если нет валидных предложений, возвращаем пустые эмбеддинги
        return batch

    embeddings = compute_sentence_embeddings(valid_sentences)

    # Приведение эмбеддингов к типу float32 для консистентности
    embeddings = embeddings.astype(np.float32)

    # Проверяем, что количество эмбеддингов совпадает с количеством предложений
    if len(embeddings) != len(valid_sentences):
        raise ValueError("Количество эмбеддингов не совпадает с количеством предложений.")
    
    # Если количество предложений после фильтрации не совпадает с исходным, корректируем выходные данные
    final_embeddings = []
    embed_idx = 0
    for sentence in sentences:
        if isinstance(sentence, str):
            final_embeddings.append(embeddings[embed_idx])
            embed_idx += 1
        else:
            final_embeddings.append(np.zeros(embeddings.shape[1], dtype=np.float32))  # Добавляем нулевые эмбеддинги для невалидных предложений

    batch[output_col] = final_embeddings
    return batch

def compute_embeddings_after_explode(batch, imput_col = 'sentences', output_col = 'sentence_embeddings'):
    return compute_embeddings(batch, imput_col = imput_col, output_col = output_col)

# Применение функции
dataset = dataset_exploded.map(compute_embeddings_after_explode, batched=True, batch_size=128)
df_embed = dataset.to_pandas()


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3243 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
df_embed.head()

,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,sentence_embeddings
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,Работает хорошо,0,"[0.1565534770488739, -0.6669323444366455, -1.0..."
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро, все целое на вид.",1,"[0.014507248997688293, -0.5217506289482117, -0..."
2,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,2,"[0.08877117931842804, 0.21360503137111664, 1.3..."
3,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...",3,"[-0.1982501894235611, -0.017847895622253418, 0..."
4,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,4,"[-0.5404854416847229, -0.10484828799962997, 0...."


In [12]:
from torch.utils.data import Dataset, DataLoader

# Определение устройства (GPU или CPU)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Перевод модели в режим FP16, если это возможно
if use_cuda:
    model_classification = model_classification.half()

# Пример данных (замените на реальные данные)
reviews = df_embed["sentences"]

# Очистка данных от некорректных значений
reviews = [str(review) for review in reviews if isinstance(review, str) and review.strip()]

# Создание кастомного Dataset для обработки отзывов
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len=128):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {key: val.flatten() for key, val in encoding.items()}

# Создаем датасет и DataLoader
dataset = ReviewDataset(reviews, tokenizer)
batch_size = 32  # Размер батча можно изменить в зависимости от объема доступной памяти GPU
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Получение предсказаний с отображением прогресса
predictions = []

from torch.cuda.amp import autocast  # Импортируем autocast для смешанной точности

for batch in tqdm(dataloader, desc="Предсказание отзывов"):
    batch = {key: val.to(device) for key, val in batch.items()}
    
    with torch.no_grad():
        with autocast():  # Используем смешанную точность
            outputs = model_classification(**batch)
            logits = outputs[0] if isinstance(outputs, tuple) else outputs.logits
            probabilities = torch.softmax(logits, dim=-1)
            batch_predictions = (probabilities[:, 1] > 0.7).cpu().numpy()  # Используем порог 0.7
            predictions.extend(batch_predictions)

# Преобразование в DataFrame, если это еще не сделано
if not isinstance(df_embed, pd.DataFrame):
    df_embed = pd.DataFrame(df_embed)

# Проверка и обработка несоответствия длины
if len(predictions) != len(df_embed):
    print(f"Warning: Length of predictions ({len(predictions)}) does not match length of index ({len(df_embed)})")
    
    # Пример: Заполнение недостающих значений
    if len(predictions) < len(df_embed):
        missing_count = len(df_embed) - len(predictions)
        predictions.extend([0] * missing_count)  # Добавляем нули в случае недостатка предсказаний

    elif len(predictions) > len(df_embed):
        predictions = predictions[:len(df_embed)]  # Обрезаем список предсказаний

# Присоединение предсказаний к датасету
df_embed['predictions'] = predictions
df_embed.head()

Предсказание отзывов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 102/102 [00:16<00:00,  6.28it/s]


,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,sentence_embeddings,predictions
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,Работает хорошо,0,"[0.1565534770488739, -0.6669323444366455, -1.0...",False
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро, все целое на вид.",1,"[0.014507248997688293, -0.5217506289482117, -0...",False
2,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,2,"[0.08877117931842804, 0.21360503137111664, 1.3...",True
3,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...",3,"[-0.1982501894235611, -0.017847895622253418, 0...",True
4,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,4,"[-0.5404854416847229, -0.10484828799962997, 0....",True


In [13]:
# Группировка и объединение предложений в одной строке для каждого уникального значения "corrected_text"
grouped_sentences = df_embed[df_embed["predictions"] == 1].groupby("corrected_text")['sentences'].apply(lambda x: " ".join(x)).reset_index()

# Переименовываем столбец для удобства
grouped_sentences = grouped_sentences.rename(columns={'sentences': 'combined_sentences'})

# Выводим результат
grouped_sentences

/opt/conda/lib/python3.10/site-packages/cudf/core/groupby/groupby.py:1343: RuntimeWarning: GroupBy.apply() performance scales poorly with number of groups. Got 523 groups. Some functions may perform better by passing engine='jit'
  warnings.warn(


,corrected_text,combined_sentences
0,)))из минуса только то что на сумке нет собачк...,из минуса только то что на сумке нет собачки н...
1,"2 раза спасали на гололедице, шлифовал на мест...","2 раза спасали на гололедице, шлифовал на мест..."
2,3из пяти помогли. Узковаты,3из пяти помогли. Узковаты
3,Авто парфюм реально суперский🔥все ришло в цел...,Авто парфюм реально суперский 🔥 все ришло в це...
4,"Анти буксы - это вещь! В снегопады спасают, чт...","Анти буксы - это вещь! В снегопады спасают, чт..."
...,...,...
518,Я не ожидал такого комплекта с арками саморезы...,Я не ожидал такого комплекта с арками саморезы...
519,"Я очень доволен ! \nВыглядят массивно, крепко....","Выглядят массивно, крепко. Сумку для инструмен..."
520,"Я просто в восторге ! Товар пришел быстро, зап...",сама уже хочу
521,"спасут, но не везде","спасут, но не везде"


In [14]:
import re
def has_multiple_words(sentence):
    # Убираем пунктуацию и разделяем предложение по пробелам
    words = re.findall(r'\b\w+\b', sentence)
    return len(words) > 1

# Группировка по 'corrected_text' и объединение предложений в 'sentences'
# df_filtered_reviews = df_embed[df_embed["predictions"] == 1].groupby(["review_full_text", "review_rating", "product", "category", "url", "corrected_text"])['sentences'].apply(lambda x: " ".join(x)).reset_index()
df_filtered_reviews = df_embed[df_embed["predictions"] == 1].drop(["Unnamed: 0", "__index_level_0__", "predictions"], axis=1)
# df_filtered_reviews["corrected_filtered_text"] = grouped_sentences['combined_sentences']
df_filtered_reviews = df_filtered_reviews[df_filtered_reviews['sentences'].apply(has_multiple_words)]
df_filtered_reviews = pd.merge(df_filtered_reviews, grouped_sentences, on='corrected_text', how='left')
df_filtered_reviews.head()

,review_full_text,review_rating,product,category,url,corrected_text,sentences,sentence_embeddings,combined_sentences
0,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,"[0.08877117931842804, 0.21360503137111664, 1.3...",Завтра буду испытывать
1,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...","[-0.1982501894235611, -0.017847895622253418, 0...","Купил на квадр для поднятия отвала, установка ..."
2,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,"[-0.5404854416847229, -0.10484828799962997, 0....",Лебёдка хорошая. Но в инструкции ни слова про ...
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Но в инструкции ни слова про сборку и креплени...,"[0.2806070148944855, 0.5317667126655579, 0.896...",Лебёдка хорошая. Но в инструкции ни слова про ...
4,Муж ещё не пробовал,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Муж ещё не пробовал,Муж ещё не пробовал,"[0.2862488627433777, 0.46963462233543396, 0.82...",Муж ещё не пробовал


In [15]:
from datasets import Dataset

def compute_embeddings_after_filter(batch, imput_col = 'combined_sentences', output_col = 'combined_sentences_embeddings'):
    return compute_embeddings(batch, imput_col = imput_col, output_col = output_col)

dataset_filtered = Dataset.from_pandas(df_filtered_reviews)

df_filtered_embed = dataset_filtered.map(compute_embeddings_after_explode, batched=True, batch_size=128).to_pandas()

Map:   0%|          | 0/1373 [00:00<?, ? examples/s]

In [16]:
# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Установка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
from loguru import logger
logger.add('./reviews_keywords/clustering.log', encoding='utf-8')

1

In [18]:
# Перевод модели в режим FP16, если это возможно
if torch.cuda.is_available():
    model_classification = model_classification.half()

def find_centroid(embeddings):
    return np.mean(embeddings, axis=0)

def extract_key_thought(sentences, embeddings = None):
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    return sentences[key_sentence_index]

def count_words(cluster_sentences):
    words = " ".join(cluster_sentences).split()
    return len(words)


In [19]:
# from transformers import pipeline
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import networkx as nx

# # Загрузка модели для резюмирования на русском языке (если она доступна)
# summarizer = pipeline("summarization", model="IlyaGusev/rugpt3medium_sum_gazeta")

# # Функции для различных методов извлечения ключевой мысли
# def extract_key_thought_method_1(sentences):
#     embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     key_sentence_index = np.argmax(similarities)
#     return sentences[key_sentence_index]

# def extract_key_thought_method_2(sentences):
#     embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     top_indices = np.argsort(similarities, axis=0)[-3:]  # Выбираем три наиболее близких предложения
#     key_thought = " ".join([sentences[idx] for idx in top_indices.flatten()])
#     return key_thought

# def extract_key_thought_method_4(sentences):
#     embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     key_sentence_index = np.argmax(similarities)
#     key_sentence = sentences[key_sentence_index]

#     # Проверяем, не пустое ли предложение и не состоит ли оно только из стоп-слов
#     if len(key_sentence.strip()) == 0:
#         return "Предложение пустое или состоит только из стоп-слов."

#     vectorizer = CountVectorizer(max_features=5, stop_words=stop_words)  # Используем русские стоп-слова
#     X = vectorizer.fit_transform([key_sentence])
    
#     if X.shape[1] == 0:
#         return f"Основной мысли нет"  # Возвращаем исходное предложение

#     key_phrases = " ".join(vectorizer.get_feature_names_out())
    
#     return f"Основная мысль: {key_phrases}."


# def extract_key_thought_method_5(sentences):
#     embeddings = get_embeddings_batchwise(sentences, model, tokenizer, batch_size=32)
#     centroid = find_centroid(embeddings)
#     similarities = cosine_similarity(embeddings, [centroid])
#     top_indices = np.argsort(similarities, axis=0)[-3:]  # Выбираем три наиболее близких предложения

#     G = nx.Graph()
#     for idx in top_indices.flatten():
#         G.add_node(sentences[idx])
#         G.add_edge('Main Idea', sentences[idx])

#     # Конвертируем граф в строковое представление
#     concept_map_str = ", ".join([str(node) for node in G.nodes if node != 'Main Idea'])
#     return f"Карта концепций: {concept_map_str}"



# def dynamic_summarization(text, summarizer, max_length_ratio=0.5, min_length=25, max_increase_step=50):
#     input_length = len(text.split())
#     # Динамически устанавливаем max_length, чтобы оно не превышало длину входного текста
#     max_length = min(int(input_length * max_length_ratio), input_length)
    
#     # Увеличиваем max_length, если это необходимо
#     while True:
#         try:
#             summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
#             return summary[0]['summary_text']
#         except ValueError as e:
#             if "Input length of input_ids" in str(e):
#                 max_length += max_increase_step
#                 if max_length > 800:  # Ограничиваем максимальное увеличение
#                     raise ValueError(f"Cannot process the text even after increasing max_length. Current max_length: {max_length}")
#             else:
#                 raise e

# def recursive_summarization(text, summarizer, max_length_ratio=0.5, min_length=25, threshold_length=200, max_increase_step=50):
#     if len(text.split()) <= threshold_length:
#         return dynamic_summarization(text, summarizer, max_length_ratio, min_length, max_increase_step)
    
#     sentences = text.split('. ')
#     mid_point = len(sentences) // 2
    
#     first_half = recursive_summarization('. '.join(sentences[:mid_point]), summarizer, max_length_ratio, min_length, threshold_length, max_increase_step)
#     second_half = recursive_summarization('. '.join(sentences[mid_point:]), summarizer, max_length_ratio, min_length, threshold_length, max_increase_step)
    
#     combined_summary = first_half + '. ' + second_half
#     return dynamic_summarization(combined_summary, summarizer, max_length_ratio, min_length, max_increase_step)

# def extract_key_thought_method_3(sentences):
#     text = " ".join(sentences)
#     return recursive_summarization(text, summarizer, max_length_ratio=0.5, min_length=25, threshold_length=200, max_increase_step=50)


# # Применение всех методов к каждому кластеру
# def apply_key_thought_extraction(df):
#     logger.info(f"extract_key_thought_method_1")
#     df['Key_Thought_Method_1'] = df['sentences'].apply(extract_key_thought_method_1)
#     logger.info(f"extract_key_thought_method_2")
#     df['Key_Thought_Method_2'] = df['sentences'].apply(extract_key_thought_method_2)
#     # logger.info(f"extract_key_thought_method_3")
#     # df['Key_Thought_Method_3'] = df['sentences'].apply(extract_key_thought_method_3)
#     logger.info(f"extract_key_thought_method_4")
#     df['Key_Thought_Method_4'] = df['sentences'].apply(extract_key_thought_method_4)
#     logger.info(f"extract_key_thought_method_5")
#     df['Key_Thought_Method_5'] = df['sentences'].apply(extract_key_thought_method_5)
#     return df



In [20]:
!export PYTHONIOENCODING=utf-8

In [21]:
from contextlib import contextmanager
@contextmanager
def suppress_output():
    devnull = os.open(os.devnull, os.O_WRONLY)
    old_stdout = os.dup(1)
    old_stderr = os.dup(2)
    os.dup2(old_stdout, 1)
    os.dup2(old_stderr, 2)
    try:
        yield
    finally:
        os.dup2(old_stdout, 1)
        os.dup2(old_stderr, 2)
        os.close(devnull)

In [22]:
import numpy as np
import pandas as pd
from hdbscan import HDBSCAN
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm

def compute_mean_distance(cluster_indices, embeddings):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = pdist(cluster_embeddings, metric='cosine')
    return np.mean(pairwise_distances)

def aggregate_embeddings(group):
    # Агрегируем эмбеддинги для предложений одного отзыва
    return np.mean(np.vstack(group["sentence_embeddings"]), axis=0)

def cluster_product_reviews_with_hdbscan(df, min_cluster_size=3, min_samples=3):
    final_clusters_list = []

    grouped = df.groupby('product')

    for product, group in tqdm(grouped, desc="Processing products"):
        embeddings = np.vstack(group["sentence_embeddings"].values)
        
        # Вычисляем косинусное расстояние
        distance_matrix = cosine_distances(embeddings)
        
        # Применяем HDBSCAN
        hdb = HDBSCAN(metric='precomputed', min_cluster_size=min_cluster_size, min_samples=min_samples)
        labels = hdb.fit_predict(distance_matrix)
        
        # Добавляем метки кластеров в DataFrame
        group['final_cluster'] = labels
        
        group['mean_distance'] = group['final_cluster'].apply(
            lambda cluster_id: compute_mean_distance(
                np.where(labels == cluster_id)[0], embeddings)
        )
        
        grouped_clusters = group.groupby('final_cluster').agg({
            'category': 'first',
            'product': 'first',
            'review_rating': 'mean',
            'url': 'first',
            'sentences': list,
            'sentence_embeddings': list,
            'mean_distance': 'first',
        }).reset_index(drop=True)
        
        grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)  
        grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(lambda x: sum([len(" ".join(s).split()) for s in x]))        
        grouped_clusters['key_thought'] = grouped_clusters.apply(
            lambda row: extract_key_thought(row['sentences'], row['sentence_embeddings']), axis=1)
        
        grouped_clusters = grouped_clusters.sort_values(by='word_count', ascending=False)
        final_clusters_list.append(grouped_clusters)
    
    return pd.concat(final_clusters_list, ignore_index=True)

# Применяем кластеризацию ко всему DataFrame
min_cluster_size = 3  # Минимальный размер кластера
min_samples = 3       # Минимальное количество точек в кластере

final_df = cluster_product_reviews_with_hdbscan(df_filtered_reviews, min_cluster_size, min_samples)
final_df = final_df.sort_values("word_count", ascending=False)
final_df

Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:06<00:00,  2.31it/s]


,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
13,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.316901,https://www.wildberries.ru/catalog/48839109/fe...,"[Можно было отлив снизу сделать, чтобы не выпа...","[[-0.2880934774875641, 0.06186602637171745, 0....",0.368512,142,5255,"2 раза спасали на гололедице, шлифовал на мест..."
24,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.450867,https://www.wildberries.ru/catalog/155565431/f...,"[Дождь пошёл покрылась, что было образуется пл...","[[-1.0747172832489014, 0.12837457656860352, -0...",0.559831,173,5117,Совсем не убирает ржавчину с машины.
19,/Автотовары/OFFroad,ПК ЛИМ / Браслеты цепи противоскольжения,4.251572,https://www.wildberries.ru/catalog/143132420/f...,"[Покупал набор из шести штук, но хватает 4 (на...","[[0.09786882996559143, 0.07219353318214417, 0....",0.506770,159,4945,"Не удобные, застежка ремня сделана не удобно, ..."
20,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.251572,https://www.wildberries.ru/catalog/143132161/f...,"[Покупал набор из шести штук, но хватает 4 (на...","[[0.06749556213617325, 0.034723978489637375, 0...",0.506064,159,4945,"Не удобные, застежка ремня сделана не удобно, ..."
21,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.565789,https://www.wildberries.ru/catalog/197410221/f...,"[Во - первых пришел совсем другой аромат ,, во...","[[-0.47253894805908203, -0.18203142285346985, ...",0.609218,152,3879,что мой чувствительный к запахам нос просто в ...
16,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,4.235772,https://www.wildberries.ru/catalog/62827233/fe...,"[В снегу действительно помогло., Не с первого ...","[[-0.19982746243476868, 0.1130455955862999, 0....",0.546376,123,3876,"Материал качество нормальное, использовать нуж..."
1,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,4.561644,https://www.wildberries.ru/catalog/124191551/f...,"[Внутри лучше разобрать и смазать, так как сма...","[[0.07086107134819031, 0.4617305099964142, -0....",0.477561,73,2625,Перестала тянуть трос после двух использовании
14,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.354167,https://www.wildberries.ru/catalog/48839109/fe...,"[Зацеп между собой не продуманный., Не во всех...","[[0.6099634170532227, 0.42926353216171265, -0....",0.647027,96,2398,Зацеп между собой не продуманный.
25,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.868421,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.197835,38,1835,"30 минут Водой не промывал, протер тряпкой."
22,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.947368,https://www.wildberries.ru/catalog/197410221/f...,"[если нос не приложить к ароматизатору …, На з...","[[0.06371931731700897, 0.32030782103538513, 0....",0.305465,38,1650,"Давно искала аромат для машины, чтобы пахло на..."


In [23]:
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist
from tqdm import tqdm
import matplotlib.pyplot as plt

def compute_mean_distance(cluster_indices, embeddings):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = pdist(cluster_embeddings, metric='cosine')
    return np.mean(pairwise_distances)

def cluster_product_reviews(df, max_distance=0.5):
    grouped = df.groupby('product')
    final_clusters_list = []

    for product, group in tqdm(grouped, desc="Processing products"):
        sentences = group["sentences"].tolist()
        embeddings = np.array(list(group["sentence_embeddings"]))
        
        distance_matrix = pdist(embeddings, metric='cosine')
        linkage_matrix = linkage(distance_matrix, method='ward')
        initial_clusters = fcluster(linkage_matrix, t=max_distance, criterion='distance')
        
        group['final_cluster'] = initial_clusters
        group['mean_distance'] = group['final_cluster'].apply(
            lambda cluster_id: compute_mean_distance(
                np.where(initial_clusters == cluster_id)[0], embeddings)
        )
        
        grouped_clusters = group.groupby('final_cluster').agg({
            'category': 'first',
            'product': 'first',
            'review_rating': 'mean',
            'url': 'first',
            'sentences': list,
            'sentence_embeddings': list,
            'mean_distance': 'first',
        }).reset_index(drop=True)
        
        grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)  
        grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(count_words)        
        grouped_clusters['key_thought'] = grouped_clusters.apply(
            lambda row: extract_key_thought(row['sentences'], row['sentence_embeddings']), axis=1)
        
        grouped_clusters = grouped_clusters.sort_values(by='word_count', ascending=False)
        final_clusters_list.append(grouped_clusters)
    
    return pd.concat(final_clusters_list, ignore_index=True)

# Применяем кластеризацию ко всему DataFrame
final_df = cluster_product_reviews(df_filtered_reviews, max_distance=0.3)
# Для вывода финального результата
final_df = final_df.sort_values("word_count", ascending=False)
display(final_df.info())
final_df[final_df["elem_count"] > 2].sort_values("product", ascending=False)

Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  7.62it/s]

<class 'cudf.core.dataframe.DataFrame'>
Index: 881 entries, 173 to 61
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   category             881 non-null    object
 1   product              881 non-null    object
 2   review_rating        881 non-null    float64
 3   url                  881 non-null    object
 4   sentences            881 non-null    list
 5   sentence_embeddings  881 non-null    list
 6   mean_distance        881 non-null    float64
 7   elem_count           881 non-null    int64
 8   word_count           881 non-null    int64
 9   key_thought          881 non-null    object
dtypes: float64(2), int64(2), list(2), object(4)
memory usage: 11.0+ MB


None

,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
751,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.642857,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.174323,14,133,Если рыхлая - 2-3 раза нужно наносить.
752,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.600000,https://www.wildberries.ru/catalog/155565431/f...,"[на следующий день высохший раствор стёр, Обра...","[[-0.16233563423156738, 0.19301316142082214, -...",0.176208,10,87,"На унитазе ржавчину убрало на раз два, чем до ..."
753,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Покупала для подготовки мангала перед покраск...,"[[-0.16269774734973907, 0.39143142104148865, 0...",0.185798,6,62,"Классное средство для удаления ржавчины, с пом..."
754,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Нанес для пробы на первый попавшийся кусок же...,"[[0.4127022922039032, 0.07070054113864899, -0....",0.244446,4,45,"Ржавчина уже хорошо въелась, пришлось нескольк..."
755,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,"[Удалял ""жучки"" на дверях авто., С крыла ржавч...","[[0.06537044048309326, 0.5978235602378845, -0....",0.150612,6,40,Отчистил этим средством всю ржавчину.
...,...,...,...,...,...,...,...,...,...,...
4,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[Лебедка хорошая лучика тянет на ура)👍👍👍, Тяне...","[[-0.2121036797761917, 0.01784498244524002, 0....",0.243116,4,32,Лебедка хорошая лучика тянет на ура)👍👍👍
5,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[Внутри лучше разобрать и смазать, так как сма...","[[0.07086107134819031, 0.4617305099964142, -0....",0.179995,4,29,"Внутри лучше разобрать и смазать, так как смаз..."
6,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[то влаги в ней не будет ,, водой сверху защит...","[[-0.10012279450893402, 0.5909180045127869, 0....",0.231133,3,21,"то влаги в ней не будет ,"
8,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,4.666667,https://www.wildberries.ru/catalog/124191551/f...,"[Только инструкции нет, Рычаг свободного хода ...","[[0.46362248063087463, 0.7040594816207886, -0....",0.245024,3,18,Вот только ремешка нет на крюк в комплекте


In [24]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm

def compute_mean_distance(cluster_indices, embeddings):
    if len(cluster_indices) < 2:
        return 0
    cluster_embeddings = embeddings[cluster_indices]
    pairwise_distances = cosine_distances(cluster_embeddings)
    return np.mean(pairwise_distances)

def cluster_product_reviews_with_kmeans(df, n_clusters=5):
    grouped = df.groupby('product')
    final_clusters_list = []

    for product, group in tqdm(grouped, desc="Processing products"):
        embeddings = np.array(list(group["sentence_embeddings"]))
        n_samples = embeddings.shape[0]
        
        if n_samples < n_clusters:
            n_clusters = max(2, n_samples)  # Корректируем количество кластеров, если данных меньше
        
        # Применяем K-means
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        labels = kmeans.fit_predict(embeddings)
        
        group['final_cluster'] = labels
        
        # Вычисляем среднее расстояние для каждого кластера
        group['mean_distance'] = group['final_cluster'].apply(
            lambda cluster_id: compute_mean_distance(
                np.where(labels == cluster_id)[0], embeddings)
        )
        
        grouped_clusters = group.groupby('final_cluster').agg({
            'category': 'first',
            'product': 'first',
            'review_rating': 'mean',
            'url': 'first',
            'sentences': list,
            'sentence_embeddings': list,
            'mean_distance': 'first',  # Добавляем среднее расстояние в итоговый DataFrame
        }).reset_index(drop=True)
        
        grouped_clusters['elem_count'] = grouped_clusters['sentences'].apply(len)
        grouped_clusters['word_count'] = grouped_clusters['sentences'].apply(count_words)
        grouped_clusters['key_thought'] = grouped_clusters.apply(
            lambda row: extract_key_thought(row['sentences'], row['sentence_embeddings']), axis=1)
        
        grouped_clusters = grouped_clusters.sort_values(by='word_count', ascending=False)
        final_clusters_list.append(grouped_clusters)
    
    return pd.concat(final_clusters_list, ignore_index=True)

# Применяем кластеризацию ко всему DataFrame
n_clusters = 20  # Задаем количество кластеров
final_df = cluster_product_reviews_with_kmeans(df_filtered_reviews, n_clusters=n_clusters)
final_df = final_df.sort_values("word_count", ascending=False)
final_df[final_df["elem_count"] > 2].sort_values("product", ascending=False)

Processing products: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:07<00:00,  2.21it/s]


,category,product,review_rating,url,sentences,sentence_embeddings,mean_distance,elem_count,word_count,key_thought
42,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.525773,https://www.wildberries.ru/catalog/155565431/f...,[всю особенность подождать сутки там желательн...,"[[-0.2752877175807953, 0.27684488892555237, 0....",0.346020,97,658,"30 минут Водой не промывал, протер тряпкой."
43,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.642857,https://www.wildberries.ru/catalog/155565431/f...,"[Дождь пошёл покрылась, что было образуется пл...","[[-1.0747172832489014, 0.12837457656860352, -0...",0.454276,70,429,"удаляет ржавчину на глазах до металла ,"
44,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.395833,https://www.wildberries.ru/catalog/155565431/f...,"[на проблемные места, за час !, Полная хрень.,...","[[0.011556755751371384, -0.09506919234991074, ...",0.656352,48,238,"если пополнить на кп при обработке ,"
39,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.893617,https://www.wildberries.ru/catalog/197410221/f...,"[если нос не приложить к ароматизатору …, На з...","[[0.06371931731700897, 0.32030782103538513, 0....",0.404765,94,634,"Давно искала аромат для машины, чтобы пахло на..."
40,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.602941,https://www.wildberries.ru/catalog/197410221/f...,"[Во - первых пришел совсем другой аромат ,, ОЦ...","[[-0.47253894805908203, -0.18203142285346985, ...",0.580406,68,384,"Купила авто парфюм, запах восхитительный 🔥"
41,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,3.096774,https://www.wildberries.ru/catalog/197410221/f...,"[во вторых исчез, за 5 дней полностью ,, в тре...","[[-0.505354106426239, 0.1778121143579483, -0.5...",0.641620,31,110,Он даже не пахнет.
36,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.128571,https://www.wildberries.ru/catalog/143132161/f...,"[Маленькие карабины не он., будет заправлять р...","[[0.2422276884317398, 0.20851214230060577, -0....",0.306248,70,518,"Не удобные, застежка ремня сделана не удобно, ..."
37,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.372549,https://www.wildberries.ru/catalog/143132161/f...,"[Удобные, но постепенно привыкаешь., как на фо...","[[0.08902470767498016, 0.07573284208774567, 0....",0.489963,51,294,Напишу позже пака не пробовали
38,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,4.315789,https://www.wildberries.ru/catalog/143132161/f...,"[Покупал набор из шести штук, но хватает 4 (на...","[[0.06749556213617325, 0.034723978489637375, 0...",0.629828,38,177,"Отличные браслеты, установил на колеса без про..."
33,/Автотовары/OFFroad,ПК ЛИМ / Браслеты цепи противоскольжения,4.142857,https://www.wildberries.ru/catalog/143132420/f...,"[Маленькие карабины не он., будет заправлять р...","[[0.23492081463336945, 0.20205345749855042, -0...",0.338690,91,670,"Не удобные, застежка ремня сделана не удобно, ..."


In [25]:
# Применяем функцию к итоговому DataFrame
final_df_filtered = apply_key_thought_extraction(final_df_filtered)

NameError: name 'apply_key_thought_extraction' is not defined

In [ ]:
final_df_filtered

,category,product,review_rating,url,sentences,mean_distance,elem_count,word_count,key_thought,Key_Thought_Method_1,Key_Thought_Method_2
2,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,"[Если не топить ее в воде полностью, то влаги ...",0.106140,3,91,Коробка с лебедкой пришла как будто её собаки ...,Коробка с лебедкой пришла как будто её собаки ...,Брад до этого меньшей мощность немного не хват...
3,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,3.333333,https://www.wildberries.ru/catalog/124191551/f...,"[Рычаг свободного хода лебедки не герметичный,...",0.161980,3,40,"Рычаг свободного хода лебедки не герметичный, ...","Рычаг свободного хода лебедки не герметичный, ...","Куда смотрите???заказывали стальной трос, а пр..."
58,/Автотовары/OFFroad,MOTORin / Расширитель колёсных арок 40 мм,3.666667,https://www.wildberries.ru/catalog/149157758/f...,"[На ниву нормально не встаёт, окантовка малень...",0.139602,3,46,"Сами расширители арок нормальные, но ставить н...","Сами расширители арок нормальные, но ставить н...","Для установки требуется частое сверление арок,..."
107,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3500lb 15...,3.200000,https://www.wildberries.ru/catalog/162316088/f...,[Просто для некоторых товарищей этот прибор до...,0.141455,5,100,"Нормальный лебёдка, но шумел, разобрался полож...","Нормальный лебёдка, но шумел, разобрался полож...","Непонятна работа тормоза, на видео сработал 2 ..."
108,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3500lb 15...,5.000000,https://www.wildberries.ru/catalog/162316088/f...,"[Хорошая лебедка, со своей задачей справляется...",0.114982,3,43,Лебёдку использую для поднятия и опускания сне...,Лебёдку использую для поднятия и опускания сне...,"Хорошая компактная лебёдка, брал для затаскива..."
131,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 4500lb 20...,3.200000,https://www.wildberries.ru/catalog/240093970/f...,[Просто для некоторых товарищей этот прибор до...,0.141455,5,100,"Нормальный лебёдка, но шумел, разобрался полож...","Нормальный лебёдка, но шумел, разобрался полож...","Непонятна работа тормоза, на видео сработал 2 ..."
132,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 4500lb 20...,5.000000,https://www.wildberries.ru/catalog/240093970/f...,"[Хорошая лебедка, со своей задачей справляется...",0.114982,3,43,Лебёдку использую для поднятия и опускания сне...,Лебёдку использую для поднятия и опускания сне...,"Хорошая компактная лебёдка, брал для затаскива..."
160,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.428571,https://www.wildberries.ru/catalog/48839109/fe...,"[Хороши когда нужно ""выскочить"" из снежного ме...",0.093537,7,144,"Хороши когда нужно ""выскочить"" из снежного мес...","Хороши когда нужно ""выскочить"" из снежного мес...","В снегопады спасают, чтобы выехать с заснеженн..."
161,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.000000,https://www.wildberries.ru/catalog/48839109/fe...,"[Можно было отлив снизу сделать, чтобы не выпа...",0.127342,11,135,Хотел вытащить катер из воды на переднем приво...,Хотел вытащить катер из воды на переднем приво...,"Ещё не пробовал, но я думаю на морозе пластик ..."
162,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.000000,https://www.wildberries.ru/catalog/48839109/fe...,"[Не во всех случаях, конечно, эти Анти буксы м...",0.117228,7,131,Зимняя резина пережевывает эти пластиковые ант...,Зимняя резина пережевывает эти пластиковые ант...,"Было бы чуть лучше, если бы шипы на пластинах ..."


In [ ]:
final_df_filtered

In [ ]:
final_df.sort_values("word_count", ascending=False)

,category,product,review_rating,url,sentences,mean_distance,elem_count,word_count,key_thought
394,/Автотовары/OFFroad,ПК ЛИМ / Браслеты цепи противоскольжения,3.928571,https://www.wildberries.ru/catalog/143132420/f...,[Уже пользовался несколько раз))) к сожалению)...,0.099308,14,315,"Протягивать ленту с той стороны колеса, одурев..."
464,/Автотовары/OFFroad,ПК ЛИМ / Цепи противоскольжения для легковых а...,3.928571,https://www.wildberries.ru/catalog/143132161/f...,[Уже пользовался несколько раз))) к сожалению)...,0.099308,14,315,"Протягивать ленту с той стороны колеса, одурев..."
304,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,3.571429,https://www.wildberries.ru/catalog/62827233/fe...,[ЗАСТРЯЛА!ДУМАЛА УЖЕ ВЕСНЫ ЖДАТЬ🙈МАШИНА СТЯЛА ...,0.110913,7,198,"Хоть производители пишет, что буксовать нельзя..."
647,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.777778,https://www.wildberries.ru/catalog/155565431/f...,[Реально штука работает намазать на ржавчину с...,0.123763,9,179,"Попробовал на машине ржа до дыры на дверке, по..."
160,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,4.428571,https://www.wildberries.ru/catalog/48839109/fe...,"[Хороши когда нужно ""выскочить"" из снежного ме...",0.093537,7,144,"Хороши когда нужно ""выскочить"" из снежного мес..."
...,...,...,...,...,...,...,...,...,...
646,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,5.000000,https://www.wildberries.ru/catalog/197410221/f...,[.],0.000000,1,1,.
771,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,[Нанёс],0.000000,1,1,Нанёс
772,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,1.000000,https://www.wildberries.ru/catalog/155565431/f...,[Го.],0.000000,1,1,Го.
773,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,1.000000,https://www.wildberries.ru/catalog/155565431/f...,[Но.],0.000000,1,1,Но.


In [ ]:
final_df.sort_values("product", ascending=False)

,category,product,review_rating,url,sentences,mean_distance,elem_count,word_count,key_thought
775,/Автотовары/OFFroad,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,4.000000,https://www.wildberries.ru/catalog/111627554/f...,[Встали как родные устанавливал на петро пикап...,0.000000,2,19,"На вид не плохие, но к сожалению на уаз профи ..."
776,/Автотовары/OFFroad,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,5.000000,https://www.wildberries.ru/catalog/111627554/f...,"[Пришли качество 🔥пока ещё не ставил, Все вста...",0.000000,2,19,Пришли качество 🔥пока ещё не ставил
647,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.777778,https://www.wildberries.ru/catalog/155565431/f...,[Реально штука работает намазать на ржавчину с...,0.123763,9,179,"Попробовал на машине ржа до дыры на дверке, по..."
648,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,4.333333,https://www.wildberries.ru/catalog/155565431/f...,"[Сам раствор мне понравился, выступала ржавчин...",0.119070,6,110,"Да и на жаре стояла машина, где были пятна ржа..."
649,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,5.000000,https://www.wildberries.ru/catalog/155565431/f...,"[Гель супер, очень порадовал, не стекает, можн...",0.137930,4,74,Прешёл по графику работает супер до этого проб...
...,...,...,...,...,...,...,...,...,...
53,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,[Не проверял],0.000000,1,2,Не проверял
54,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,[Установлю],0.000000,1,1,Установлю
55,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,5.000000,https://www.wildberries.ru/catalog/124191551/f...,[be/X6alkqqmbiQ?si=_dV1lFSghhipjst6],0.000000,1,1,be/X6alkqqmbiQ?si=_dV1lFSghhipjst6
0,/Автотовары/OFFroad,Autobrand_AED / Дополнительная led фара 30w с ...,2.000000,https://www.wildberries.ru/catalog/90401367/fe...,"[Но через месяц сгорел один диод, второй ещё ч...",0.000000,1,13,"Но через месяц сгорел один диод, второй ещё че..."


In [ ]:
# Удаление записей с word_count <= 10 и ключевой мыслью менее 3 символов
final_result = final_result[((final_result['word_count'] > 10) & (final_result['key_thought'].str.len() > 5))]
final_result

NameError: name 'final_result' is not defined

In [ ]:
final_result.to_csv("./reviews_keywords/feedbackfueltest.csv")